In [1]:
# @title 1.1 Install Condacolab (< 1min)
%%time

! pip install -q condacolab
import condacolab
condacolab.install()

#@markdown *Kernel will restart automatically after this installation*
#@markdown
#@markdown *Keep connected to the same runtime and proceed to the next code block*


⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...
CPU times: user 459 ms, sys: 205 ms, total: 663 ms
Wall time: 19.3 s


In [1]:
# @title 1.2 Install Packages and Data (~ 5min)
%%time

# Install Reduce2 (cctbx-base)
! conda install cctbx-base


# Install Prody and py3Dmol
! pip install prody py3Dmol


# Download Phenix Project geostd (restraint) Library
goestd_repo = "https://github.com/phenix-project/geostd.git"
! git clone {goestd_repo}


# Install Vina, Meeko (develop branch) and Dependencies
! conda install numpy scipy rdkit vina
! git clone --single-branch --branch develop https://github.com/forlilab/Meeko.git
! cd Meeko; pip install --use-pep517 -e .; cd ..


# Install Scrubber (develop branch)
! git clone --single-branch --branch develop https://github.com/forlilab/scrubber.git
! cd scrubber; pip install --use-pep517 -e .; cd ..


# Download Vina Executables
! wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
! mv vina_1.2.5_linux_x86_64 vina; chmod +x vina;
! wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
! mv vina_split_1.2.5_linux_x86_64 vina_split; chmod +x vina_split


# Download AutoGrid and ADGPU Executable
! git clone -n --depth=1 --filter=tree:0 \
  https://github.com/rwxayheee/Colabs.git; cd Colabs; \
  git sparse-checkout set --no-cone Compiled_for_Colab; git checkout
! cp /content/Colabs/Compiled_for_Colab/AutoGird_v4.2.6/autogrid4 autogrid4; chmod +x autogrid4
! cp /content/Colabs/Compiled_for_Colab/AutoDock-GPU_v1.5.3/autodock_gpu_128wi adgpu; chmod +x adgpu

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cctbx-base


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.84             |  py310hc51659f_0         2.7 MB  conda-forge
    brotli-1.1.0               |       hd590300_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hd590300_1          19 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    cctbx-base-2024.8          |  py310h159de0f_0        61.0 MB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    contou

In [2]:
# @title 1.3 Import Modules & Helper Functions (< 1s)
%%time

# Import modules
import sys, platform
from prody import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit, py3Dmol
print("rdkit version:", rdkit.__version__)
print("py3Dmol version:", py3Dmol.__version__)
from ipywidgets import interact, IntSlider
import ipywidgets, copy


# Helper functions
def locate_file(from_path = None, query_path = None, query_name = "query file"):

    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]


def confgen_from_smiles(smi = None, outputSDF = "ligand.sdf", numConfs = 50, maxIters = 5000):

    def which_conformer(opt_res):

      normal_res = [x for x in opt_res if x[0]==0]

      if len(normal_res)==0: # all opt failed
          return -1
      else:
          min_eng = min([x[1] for x in normal_res])
          for conf_id,r in enumerate(opt_res):
            if (r[0]==0 and r[1]==min_eng):
                return conf_id

    if smi is None:
      raise ValueError("Must specify smi")

    m = Chem.MolFromSmiles(smi)

    mh = Chem.AddHs(m)

    params = AllChem.ETKDGv3()
    params.numThreads = 0

    cids = AllChem.EmbedMultipleConfs(mh, numConfs, params)
    res = AllChem.MMFFOptimizeMoleculeConfs(mh, numThreads = 0, maxIters = maxIters)

    minconf = which_conformer(res)
    if minconf!=-1:
        with Chem.SDWriter(outputSDF) as w:
          w.write(mh, confId=minconf)
    else:
      raise RuntimeError("Optimization didn't converge. No conformer/SDF will be written")

# Commandline scripts
scrub = locate_file(from_path = Path("/usr/local/bin"), query_path = "scrub.py", query_name = "scrub.py")
mk_prepare_ligand = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_ligand.py", query_name = "mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_receptor.py", query_name = "mk_prepare_receptor.py")
mk_export = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_export.py", query_name = "mk_export.py")


# Locate reduce2 in conda install prefix
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
env_path = Path("/usr/local") # default conda install prefix on Colab
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")


# Locate geostd in current path
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

rdkit version: 2023.09.6
py3Dmol version: 2.4.0
using scrub.py at:
/usr/local/bin/scrub.py

using mk_prepare_ligand.py at:
/usr/local/bin/mk_prepare_ligand.py

using mk_prepare_receptor.py at:
/usr/local/bin/mk_prepare_receptor.py

using mk_export.py at:
/usr/local/bin/mk_export.py

using reduce2.py at:
/usr/local/lib/python3.10/site-packages/mmtbx/command_line/reduce2.py

using geostd at:
/content/geostd

CPU times: user 712 ms, sys: 103 ms, total: 815 ms
Wall time: 1.42 s


In [9]:
# @title # 2.1 [Reactive Docking] Conformer Generation of Pre-reaction Ligand (< 5s)
%%time

#@markdown ## (1) Conformer Generation with RDKit

# Specify ligand Smiles
# @markdown *Input*
ligand_Smiles = "c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)([O-])[O-])O)O)N" #@param {type:"string"}

# Write 3D conformer to ligand SDF
# @markdown *Output*
ligandSDF = "AMP.sdf" #@param {type:"string"}
confgen_from_smiles(smi = ligand_Smiles, outputSDF = ligandSDF)


# Visualization with py3Dmol
view = py3Dmol.view()
view.addModel(open(ligandSDF, 'r').read(),'sdf')
view.zoomTo()
view.setBackgroundColor('white')
view.addStyle({'stick': {'colorscheme':'yellowCarbon'}})
view.show()


#@markdown ---
#@markdown ## (2) Reactive Ligand Preparation with Meeko

# Prepare ligand PDBQT
# @markdown *Options*
reactive_smarts = "COP(=O)([O-])[O-]" #@param {type:"string"}
reactive_smarts_idx = "3" #@param {type:"string"}

# @markdown *Output*
ligandPDBQT = "AMP.pdbqt" #@param {type:"string"}
! python {mk_prepare_ligand} -i {ligandSDF} --reactive_smarts "{reactive_smarts}" --reactive_smarts_idx {reactive_smarts_idx} -o {ligandPDBQT}


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

CPU times: user 3.92 s, sys: 18.2 ms, total: 3.93 s
Wall time: 4.51 s


In [14]:
# @title # 2.2 [Reactive Docking] Receptor Preparation (< 30s)
%%time

#@markdown ## (1) Add Hydrogens to Receptor with Reduce2

# @markdown *Input*
# Download PDB file
pdb_token = "3kgd" #@param {type:"string"}
! curl "http://files.rcsb.org/view/{pdb_token}.pdb" -o "{pdb_token}.pdb"


# Export receptor atoms
atoms_from_pdb = parsePDB(pdb_token)
# @markdown *ProDy Options*
receptor_selection = "chain A and not water and not hetero and not resname AMP" #@param {type:"string"}
receptor_atoms = atoms_from_pdb.select(receptor_selection)
# @markdown *Prody Output*
prody_receptorPDB = "3kgd_receptor_atoms.pdb" #@param {type:"string"}
writePDB(prody_receptorPDB, receptor_atoms)


# Add CRYST1 card (temporarily required for reduce2)
# @markdown *Reduce Input*
reduce_inputPDB = "3kgd_receptor.pdb" #@param {type:"string"}
! cat <(grep "CRYST1" "{pdb_token}.pdb") {prody_receptorPDB} > {reduce_inputPDB}


# Run reduce2
# @markdown *Reduce Options*
reduce_opts = "approach=add add_flip_movers=True" #@param {type:"string"}
! export MMTBX_CCP4_MONOMER_LIB="{geostd_path}"; python {reduce2} {reduce_inputPDB} {reduce_opts}
# Default name of reduce output...
prepare_inPDB = "3kgd_receptorFH.pdb" #@param {type:"string"}

#@markdown ---
#@markdown ## (2) Receptor Preparation with Meeko

# @markdown *Center the box at...*
# Specify ligand
atoms_from_pdb = parsePDB(pdb_token)
ligand_selection = "chain A and resname AMP" #@param {type:"string"}
ligand_atoms = atoms_from_pdb.select(ligand_selection)
center_x, center_y, center_z = calcCenter(ligand_atoms)


# @markdown *For future reference, export ligand's original position to...*
prody_ligandPDB = "LIG.pdb" #@param {type:"string"}
writePDB(prody_ligandPDB, ligand_atoms)


# @markdown *Set size of the box by...*
# Padding in each dimension
padding = 8.0 #@param {type:"raw"}


# @markdown *Options*
args = ""
allow_bad_res = True #@param {type:"boolean"}
if allow_bad_res:
  args += "--allow_bad_res "
reactive_name_specific = "A:309=NE2" #@param {type:"string"}


# @markdown *Output*
# Prepare Receptor
prepare_outPDBQT = "3kgd_receptorFH.pdbqt" #@param {type:"string"}
! python {mk_prepare_receptor} --pdb {prepare_inPDB} --reactive_name_specific {reactive_name_specific} -o {prepare_outPDBQT} --box_enveloping {prody_ligandPDB} --padding {padding} {args}


# Visualization with py3Dmol
def Receptor3DView(receptorPDB = None, boxPDB = None, ligPDB = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    view.addModel(open(boxPDB, 'r').read(),'pdb')
    view.addStyle({'stick': {}})
    view.zoomTo()

    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligPDB is not None:
      view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    return view

Receptor3DView(receptorPDB = prepare_inPDB, \
               boxPDB = prepare_outPDBQT.replace('.pdbqt', '_rigid.box.pdb'), \
               ligPDB = prody_ligandPDB).show()

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2003k    0 2003k    0     0  6958k      0 --:--:-- --:--:-- --:--:-- 6980k


@> PDB file is found in working directory (3kgd.pdb).
@> 11804 atoms and 1 coordinate set(s) were parsed in 0.16s.


Starting /usr/local/lib/python3.10/site-packages/mmtbx/command_line/reduce2.py
on Tue Sep 17 23:59:52 2024 by root

Processing files:
-------------------------------------------------------------------------------

  Found model, 3kgd_receptor.pdb

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    approach=add
    add_flip_movers=True

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  data_manager {
    model {
      file = "3kgd_receptor.pdb"
    }
    default_model = "3kgd_receptor.pdb"
  }
  add_flip_movers = True


Starting job
Writing model output to 3kgd_receptorFH.pdb

                       ----------Loading Model----------                       


                      ----------Adding Hydrogens----------                     

Number of hydrogen atoms added to the input model: 2560 


The followi

@> PDB file is found in working directory (3kgd.pdb).
@> 11804 atoms and 1 coordinate set(s) were parsed in 0.14s.


templates=<meeko.linked_rdkit_chorizo.ResidueChemTemplates object at 0x7862f3e02e00>
[00:00:13] product atom-mapping number 11 not found in reactants.
[00:00:13] product atom-mapping number 12 not found in reactants.
[00:00:13] product atom-mapping number 11 not found in reactants.
[00:00:13] product atom-mapping number 12 not found in reactants.
[00:00:13] product atom-mapping number 13 not found in reactants.

Flexible residues:
chain resnum is_reactive reactive_atom
    A    309        True           NE2
reactive_flexres={'A:309'}

For reactive docking, pass the configuration file to AutoDock-GPU:
    autodock_gpu -C 1 --import_dpf 3kgd_receptorFH.reactive_config --flexres 3kgd_receptorFH_flex.pdbqt -L <ligand_filename>


Files written:
     3kgd_receptorFH_flex.pdbqt <-- flexible receptor input file
    3kgd_receptorFH_rigid.pdbqt <-- static (i.e., rigid) receptor input file
     boron-silicon-atom_par.dat <-- atomic parameters for B and Si (for autogrid)
      3kgd_receptorFH_rigi

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

CPU times: user 594 ms, sys: 21.4 ms, total: 615 ms
Wall time: 24.1 s


In [15]:
# @title # 2.2 (3) Compute Gird Maps (< 15s)
%%time

! ./autogrid4 -p {prepare_outPDBQT.replace('.pdbqt', '_rigid.gpf')}

Streaming output truncated to the last 5000 lines.
Atom no. 2512, " CB " at (5.476, -26.990, 3.977),  was assigned atom type "C" (rec_index= 1, atom_type= 1).
Atom no. 2513, " OG " at (5.675, -26.739, 2.603),  was assigned atom type "OA" (rec_index= 5, atom_type= 5).
Atom no. 2514, " HG " at (6.474, -26.522, 2.463),  was assigned atom type "HD" (rec_index= 0, atom_type= 0).
Atom no. 2515, " H  " at (4.190, -25.904, 5.944),  was assigned atom type "HD" (rec_index= 0, atom_type= 0).
Atom no. 2516, " C  " at (5.071, -22.899, 1.241),  was assigned atom type "C" (rec_index= 1, atom_type= 1).
Atom no. 2517, " O  " at (4.608, -21.899, 0.688),  was assigned atom type "OA" (rec_index= 5, atom_type= 5).
Atom no. 2518, " CA " at (6.034, -22.763, 2.443),  was assigned atom type "C" (rec_index= 1, atom_type= 1).
Atom no. 2519, " N  " at (6.287, -24.028, 3.154),  was assigned atom type "N" (rec_index= 3, atom_type= 3).
Atom no. 2520, " CB " at (7.327, -22.190, 1.893),  was assigned atom type "C" (re

In [25]:
# @title # 2.3 [Reactive Docking] Docking with ADGPU (< 5s)
%%time
output_resnam = "AMP" #@param {type:"string"}
! touch empty
! ./adgpu --lfile {ligandPDBQT} --flexres {prepare_outPDBQT.replace('.pdbqt', '_flex.pdbqt')} --ffile {prepare_outPDBQT.replace('.pdbqt', '_rigid.maps.fld')} --import_dpf {prepare_outPDBQT.replace('.pdbqt', '.reactive_config')} --resnam {output_resnam}

AutoDock-GPU version: v1.5.3

Running 1 docking calculation

Cuda device:                              Tesla T4
Available memory on device:               14999 MB (total: 15102 MB)

CUDA Setup time 0.209721s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 2040163
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.021782s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 2040163 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |    1.34 kcal/mol |   -1.93 +/-    1.03 kcal/mol |       5 |   -3.51 kcal/mol
          5 |        62821 |    1.16 kcal/mol |   -6.37 +/-    2.17 kcal/mol |     753 |  -13.75 kcal/mol
     

In [26]:
# @title # 2.4 [Covalent Docking] Export and Visualize Docked Poses (~ 1s)
%%time

# Export Docked Poses
#@markdown *Export docked poses to...*
dock_outSDF = "3kgd_AMP_adgpu_out.sdf" #@param {type:"string"}
! python {mk_export} {output_resnam}.dlg -o {dock_outSDF}

#@markdown *Visualize docked poses with...*
# Previously Generated Receptor Files
receptorPDB = "3kgd_receptorFH.pdb" #@param {type:"string"}
boxPDB = "3kgd_receptorFH_rigid.box.pdb" #@param {type:"string"}
refligPDB = 'LIG.pdb' #@param {type:"string"}
reflig_resn = 'AMP' #@param {type:"string"}

# Visualize Docked Poses
def Complex3DView(view, ligmol = None, refligPDB = None, reflig_resn = None):

    new_viewer = copy.deepcopy(view)

    mblock = Chem.MolToMolBlock(ligmol)
    new_viewer.addModel(mblock, 'mol')
    new_viewer.addStyle({'hetflag': True}, {"stick": {'colorscheme': 'greenCarbon'}})

    if refligPDB is not None and reflig_resn is not None:
      new_viewer.addModel(open(refligPDB, 'r').read(), 'pdb')
      new_viewer.addStyle({'resn': reflig_resn}, {"stick": {'colorscheme': 'magentaCarbon', 'opacity': 0.8}})

    return new_viewer


confs = Chem.SDMolSupplier(dock_outSDF)

def conf_viewer(idx):
    mol = confs[idx]
    return Complex3DView(Receptor3DView(receptorPDB = receptorPDB, boxPDB = boxPDB), \
                         mol, \
                         refligPDB = refligPDB, reflig_resn = reflig_resn).show()


interact(conf_viewer, idx=ipywidgets.IntSlider(min=0, max=len(confs)-1, step=1))

interactive(children=(IntSlider(value=0, description='idx', max=19), Output()), _dom_classes=('widget-interact…

In [27]:
# Download Zipped Output
! mkdir output; cp * output
! zip -r output.zip output

cp: -r not specified; omitting directory 'Colabs'
cp: -r not specified; omitting directory 'geostd'
cp: -r not specified; omitting directory 'Meeko'
cp: -r not specified; omitting directory 'output'
cp: -r not specified; omitting directory 'sample_data'
cp: -r not specified; omitting directory 'scrubber'
  adding: output/ (stored 0%)
  adding: output/3kgd_receptorFH_rigid.e.map (deflated 68%)
  adding: output/empty (stored 0%)
  adding: output/adgpu (deflated 62%)
  adding: output/boron-silicon-atom_par.dat (deflated 39%)
  adding: output/vina_split (deflated 65%)
  adding: output/3kgd_receptorFH_rigid.A.map (deflated 59%)
  adding: output/3kgd_receptorFH_rigid.I.map (deflated 57%)
  adding: output/3kgd_receptorFH_rigid.HD.map (deflated 65%)
  adding: output/3kgd_receptorFH_rigid.d.map (deflated 73%)
  adding: output/vina (deflated 63%)
  adding: output/3kgd_receptorFH_rigid.P.map (deflated 58%)
  adding: output/LIG.pdb (deflated 70%)
  adding: output/3kgd_receptorFH_flex.pdbqt (deflat